In [3]:
from pathlib import Path
import numpy as np
import os
import pandas as pd
import re
import time

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
print(sys.path)

['/home/hieutt/UniCon/notebooks', '/home/hieutt/miniconda3/envs/torchtf/lib/python39.zip', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [5]:
import os
import pandas as pd

# Định nghĩa đường dẫn thư mục chứa các thư mục tấn công
input_dir = '../data/can-ml/2017-subaru-forester/post-attack-labeled'  # Thay bằng đường dẫn đến thư mục của bạn
output_dir = '../data/can-ml/2017-subaru-forester/merged'  # Thay bằng đường dẫn thư mục đầu ra cho các file gộp

# Tạo thư mục đầu ra nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)

# Danh sách các loại tấn công (tên thư mục)
attack_types = ['combined-attacks', 'DoS-attacks', 'fuzzing-attacks', 'gear-attacks',
                'interval-attacks', 'rpm-attacks', 'speed-attacks', 'standstill-attacks', 'systematic-attacks']

# Vòng lặp qua từng loại tấn công
for attack_type in attack_types:
    attack_folder = os.path.join(input_dir, attack_type)
    
    # Kiểm tra nếu thư mục tồn tại
    if not os.path.isdir(attack_folder):
        print(f"Folder {attack_folder} does not exist, skipping.")
        continue
    
    # Khởi tạo danh sách để lưu DataFrame
    df_list = []
    
    # Vòng lặp qua từng file CSV trong thư mục
    for file_name in os.listdir(attack_folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(attack_folder, file_name)
            
            # Đọc file CSV
            df = pd.read_csv(file_path)
            
            # Tiền xử lý dữ liệu
            # Chuyển đổi arbitration_id về số nguyên (decimal) nếu có giá trị hợp lệ
            df['arbitration_id'] = df['arbitration_id'].apply(lambda x: int(x, 16) if isinstance(x, str) else x)
            
            # Chuyển đổi data_field về dạng binary với độ dài cố định 64 bit nếu là chuỗi hợp lệ
            df['data_field'] = df['data_field'].apply(lambda x: bin(int(x, 16))[2:].zfill(64) if isinstance(x, str) else x)
            
            # Loại bỏ các dòng có bất kỳ giá trị nào không hợp lệ
            df = df[['timestamp', 'arbitration_id', 'data_field', 'attack']].dropna()

            # Thêm DataFrame vào danh sách
            df_list.append(df)
    
    # Gộp tất cả DataFrame trong danh sách
    merged_df = pd.concat(df_list, ignore_index=True)
    
    # Lưu DataFrame đã gộp vào file CSV mới
    attack_type_cleaned = attack_type.replace("-attacks", "")
    output_file = os.path.join(output_dir, f'{attack_type_cleaned}.csv')
    merged_df.to_csv(output_file, index=False)
    
    print(f"Merged {attack_type_cleaned} saved to {output_file}")


Merged combined saved to ../data/can-ml/2017-subaru-forester/merged/combined.csv
Merged DoS saved to ../data/can-ml/2017-subaru-forester/merged/DoS.csv
Merged fuzzing saved to ../data/can-ml/2017-subaru-forester/merged/fuzzing.csv
Merged gear saved to ../data/can-ml/2017-subaru-forester/merged/gear.csv
Merged interval saved to ../data/can-ml/2017-subaru-forester/merged/interval.csv
Merged rpm saved to ../data/can-ml/2017-subaru-forester/merged/rpm.csv
Merged speed saved to ../data/can-ml/2017-subaru-forester/merged/speed.csv
Merged standstill saved to ../data/can-ml/2017-subaru-forester/merged/standstill.csv
Merged systematic saved to ../data/can-ml/2017-subaru-forester/merged/systematic.csv


In [4]:
# Define the directory path
directory_path = '../data/can-ml/2017-subaru-forester/merged/fuzzing.csv'

df = pd.read_csv(directory_path)
# Display the first few rows of the combined DataFrame
filtered_df = df[df['attack'] != 0]
filtered_df

,timestamp,arbitration_id,data_field,attack
9205,1.672532e+09,1576,0000000000000000000000000000000000000000000000...,1
9209,1.672532e+09,1120,1111000001101110010110100011000101011000010010...,1
9212,1.672532e+09,322,0011000100101001001101110000101101010011101001...,1
9214,1.672532e+09,1574,NaN,1
9216,1.672532e+09,1957,1011001111010110001000010011000011100111010001...,1
...,...,...,...,...
2353942,1.672532e+09,394,0001011111011000110101100000111111100100110000...,1
2353943,1.672532e+09,628,0000100001001000000110100011111001101010011010...,1
2353946,1.672532e+09,27,0000000001111110001101101110110100011100010010...,1
2353949,1.672532e+09,642,0000000000000000000000000000000000000000000000...,1


In [20]:
as_strided = np.lib.stride_tricks.as_strided
output_shape = (392625, 32)


label_new = np.lib.stride_tricks.sliding_window_view(df.attack.values, window_shape=32)[::16]


# label = as_strided(df.attack.values, output_shape, (1 * 16, 1))
print(label_new)
# df['label'] = df['label'].apply(lambda x: 1 if any(x) else 0)
# df

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
